In [1]:
import asyncio
from azure.eventhub.aio import EventHubProducerClient
from azure.eventhub import EventData
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import nltk
import os
from azure.eventhub.aio import EventHubProducerClient, EventHubSharedKeyCredential
from textblob import TextBlob, Word, Blobber
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')

[nltk_data] Downloading package punkt to /Users/amanakmsd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/amanakmsd/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to /Users/amanakmsd/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
def getblobsentiment(text):
    blob = TextBlob(text)
    i=0.00
    for sentence in blob.sentences:
        i=i+sentence.sentiment.polarity
        #print(sentence.sentiment.polarity)
        if i >0: 
            return 1
        elif i<0:
            return -1
        else:
            return 0

async def run (text):
    producer = EventHubProducerClient.from_connection_string(conn_str="Endpoint=sb://twittersentitmenteventhub.servicebus.windows.net/;SharedAccessKeyName=TweetSAS;SharedAccessKey=zxu4/wVPEDLU3jMgX9BPX6tP155LikPjPpjzHwC4d7c=;EntityPath=twittereventhub",eventhub_name="twittereventhub")
    print(producer)
    
    async with producer:
    # Create a batch.
        print(text)
        event_data_batch = await producer.create_batch()
        event_data_batch.add(EventData(text))
        await producer.send_batch(event_data_batch)

In [4]:
class listener(StreamListener):
      
        def on_data(self, data): 
            loop = asyncio.get_event_loop()

            t=json.loads(data)

            d1 = {"text":t["text"] ,"sentiment": getblobsentiment(t["text"]) ,"created_at" :t["created_at"]  }
            print (type(data))     
            loop.run_until_complete(run(json.dumps(d1)))
            print(d1)

            return(True)

        def on_error(self, status):
            print(status)
        
ckey='CONSUMER_KEY'
csecret='CONSUMER_SECRET'
atoken='ACCESS_KEY'
asecret='ACCESS_SECRET'

auth = OAuthHandler(ckey, csecret) 
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
#twitterStream.filter(track=["positive"])
twitterStream.filter(track=["Trump"])

<class 'str'>
{"text": "RT @camrynbaylee: On International Women's Day, I would like to remind the world that President Donald J. Trump had more women as top advis\u2026", "sentiment": 1, "created_at": "Mon Mar 08 16:59:19 +0000 2021"}
{'text': "RT @camrynbaylee: On International Women's Day, I would like to remind the world that President Donald J. Trump had more women as top advis…", 'sentiment': 1, 'created_at': 'Mon Mar 08 16:59:19 +0000 2021'}
<class 'str'>
{"text": "Not too sure why those Capitol Raiders were charged and imprisoned, when the HK Violent Rioters &amp; Murderers are con\u2026 https://t.co/DfEXmRKTfv", "sentiment": -1, "created_at": "Mon Mar 08 16:59:19 +0000 2021"}
{'text': 'Not too sure why those Capitol Raiders were charged and imprisoned, when the HK Violent Rioters &amp; Murderers are con… https://t.co/DfEXmRKTfv', 'sentiment': -1, 'created_at': 'Mon Mar 08 16:59:19 +0000 2021'}
<class 'str'>
{"text": "RT @manstol1: @davidschneider A year ago today, David Schnei

ProtocolError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))